In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import requests
import shutil
import chime; chime.theme('mario')
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from datetime import date, datetime
from IPython.core.debugger import Pdb #Pdb().set_trace()


# Define your login credentials
username = ""
password = ""

In [ ]:
update_file = "new_file.xlsx"

In [ ]:
def read_not_published_data(file):
    df = pd.read_excel(f'data/{file}')
    try:
        not_published = df[df["published"]==0].reset_index(drop=True) #Not published yet
        not_published = not_published.rename(columns=str.lower)
        df = not_published.copy()
    except:
        pass
    return df


not_published = read_not_published_data( update_file )
# Self note, make warning for false not being the only value

In [ ]:
def paraphrase_with_TOOL(not_published):
    
    not_published_updated = not_published.copy()
    not_published_updated["paraphrased title"] = pd.Series(np.empty(0, dtype=str))
    not_published_updated["paraphrased description"] = pd.Series(np.empty(0, dtype=str))
    
    # Creatting settings for log in 
    login_url = ""
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options)
    driver.implicitly_wait(60)

    # Initialization of session
    driver.get(login_url)
    cookies = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    ActionChains(driver).move_to_element(cookies).click(cookies).perform()
    usr_box = driver.find_element("name", 'username')
    pas_box = driver.find_element("name", 'password')
    login_button = driver.find_element(By.XPATH, '//*[@id="loginContainer"]/div[2]/div[2]/div[5]/button')
    ActionChains(driver).move_to_element(usr_box).click(usr_box).send_keys(username).perform()
    ActionChains(driver).move_to_element(pas_box).click(pas_box).send_keys(password).perform()
    ActionChains(driver).move_to_element(login_button).click(login_button).perform()
    
    # Ignoring unnecessary info
    try:
        done_button = driver.find_element(By.CSS_SELECTOR, '.css-78lmr4')
        ActionChains(driver).move_to_element(done_button).click(done_button).perform()
    except:
        pass

    # TASK TO BE DONE AT THE WEBPAGE!!!
    # Make sure that the "Cretative" and the "Expand" buttons are visible at the bar: 
    # Then, write "c" and press "enter" in keyboard
    Pdb().set_trace()
    
    # Getting location of important buttons
    creative = driver.find_element(By.XPATH, '//*[@id="Paraphraser-mode-tab-5"]')
    expand = driver.find_element(By.XPATH, '//*[@id="Paraphraser-mode-tab-6"]')
    paraphrased_box  = driver.find_element(By.XPATH, '//*[@id="paraphraser-output-box"]')
    text_box = driver.find_element(By.CSS_SELECTOR, ".css-1pc8vv7")

    for i in range(len(not_published_updated)):
        # Paraphrasing for case study title
        _control = paraphrased_box.text
        original_text_title = not_published_updated["Innovation_Title"][i] 
        ActionChains(driver).move_to_element(creative).click(creative).perform()
        text_box.send_keys(original_text_title)
        # --  To paraphrase
        time.sleep(5) # required, otherwise webpage bugs
        ActionChains(driver).move_to_element(text_box).click(text_box).perform()
        ActionChains(driver).key_down(Keys.CONTROL).send_keys(Keys.ENTER).perform()
        ActionChains(driver).move_to_element(paraphrased_box).click(paraphrased_box).perform()
        driver.find_element(By.CSS_SELECTOR, '.css-10dohqv') # sanity check: text finished processing
        time.sleep(5)
        _control = paraphrased_box.text
        (# -- Clean search box
            ActionChains(driver).move_to_element(text_box).click(text_box)
                .key_down(Keys.CONTROL).send_keys('A').key_up(Keys.CONTROL)
                .send_keys(Keys.DELETE).perform()
        )

        if len(_control) == 0:
            chime.warning()
            print("\n\n\n")
            print("Please, fix title. Afterwards, CLEAN THE TEXT BOX. When finished, press c")
            Pdb().set_trace()
            _control = paraphrased_box.text
        
        not_published_updated.loc[i, "paraphrased title"] = _control
        
        # Paraphrasing for case study description
        _control = paraphrased_box.text
        original_text_description = not_published_updated["Innovation_Description"][i] 
        ActionChains(driver).move_to_element(expand).click(expand).perform()
        ActionChains(driver).move_to_element(text_box).click(text_box).perform()
        text_box.send_keys(original_text_description)
        # --  To paraphrase
        time.sleep(5) # required, otherwise webpage bugs
        ActionChains(driver).move_to_element(text_box).click(text_box).perform()
        ActionChains(driver).key_down(Keys.CONTROL).send_keys(Keys.ENTER).perform()
        ActionChains(driver).move_to_element(paraphrased_box).click(paraphrased_box).perform()
        driver.find_element(By.CSS_SELECTOR, '.css-10dohqv') # sanity check: text finished processing
        time.sleep(7)
        _control = paraphrased_box.text
        (# -- Clean search box
            ActionChains(driver).move_to_element(text_box).click(text_box)
                .key_down(Keys.CONTROL).send_keys('A').key_up(Keys.CONTROL)
                .send_keys(Keys.DELETE).perform()
        )
        if len(_control) == 0:
            chime.warning()
            print("\n\n\n")
            print("Please, fix description. Afterwards, CLEAN THE TEXT BOX. When finished, press c")
            Pdb().set_trace()
            _control = paraphrased_box.text

        not_published_updated.loc[i, "paraphrased description"] = _control

    chime.success()
        
    return not_published_updated

In [ ]:
not_published_updated = paraphrase_with_TOOL(not_published)

In [ ]:
not_published_updated.head()

In [ ]:
not_published_updated.to_csv(f"data/{update_file}__UPDATED.csv", index=False)